In [1]:
processed%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import requests, datetime
from datetime import date, timedelta
from utils import *
from newspaper import Article
from IPython.display import clear_output
from urllib.parse import urlencode, quote_plus
from tqdm.notebook import tqdm
import random
from IPython.display import Image


from loguru import logger
logger.add("process.log")

import pickle

In [222]:
@logger.catch
def search_arquivo_news(query: str, _from: datetime, _to: datetime, websites: [str], max_items=2000, _type="html", fields="title,tstamp,originalURL,linkToNoFrame"):
    params = {
        "q": '"%s"' % query,
        "from": arquivo_date(_from),
        "to": arquivo_date(_to),
        "siteSearch": ",".join(websites),
        "fields": fields,
        "type": _type,
        "maxItems": max_items,
        "itemsPerSite": max_items//len(websites),
    }
    
    # itemsPerSite does not work
    return query_arquivo(params, "https://arquivo.pt/textsearch", attempts=10)

@logger.catch
def get_random_arquivo_image(query: str, _from: datetime = datetime(2018,10,1), _to: datetime = date.today(), show=False):
    params = {
        "q": '"%s"' % query,
        "from": arquivo_date(_from),
        "to": arquivo_date(_to)
    }
    
    response = query_arquivo(params, "https://arquivo.pt/imagesearch", attempts=10)
    if not response: return None
    
    chosen_index = random.randint(0, len(response) - 1)
    random_image = response[chosen_index]['imgLinkToArchive']
    
    if show: display(Image(url=random_image, width=300))
    return random_image


@logger.catch
def query_arquivo(params, endpoint, timeout=30, attempts=1):
    logger.info("[FETCHING] %s" % endpoint + '?' + urlencode(params, quote_via=quote_plus))
    
    r = try_request(endpoint, params, timeout, attempts)
    if not r: return []
    json_response = r.json()
    return json_response["response_items"] if "response_items" in json_response else json_response["responseItems"]

@logger.catch
def process_news_piece(arquivo_item):
    
    # Clean news piece (returns None if some scrapping error occured)
    processing_article = clean_response_item(arquivo_item.copy())
    if processing_article is None:
        return {"valid": False}
    
    # Check if the arquivo link is up 
    html = try_request(processing_article["url"])
    if not html:
        if html == False:  # resource will never be available
            processing_article["valid"] = False
            logger.error("%s will never be available" % (processing_article["url"]))
        return
    
    try:
        # Process using Newspaper3k Package
        article_obj = Article(processing_article["url"], _language="pt")
        article_obj.download(input_html=html.text)
        article_obj.parse()
        article_obj.nlp()
        
        processing_article["text"] = assert_valid_article(article_obj)
        processing_article["image"] = article_obj.top_image
        processing_article["summary"] = article_obj.summary
        processing_article["summary_entities"], duplicates = organize_entities(processing_article["summary"])
        processing_article["title_entities"], _ = organize_entities(processing_article["title"], duplicates)
        processing_article["valid"] = True
        
    except Exception as e:
#         print(e.with_traceback())
        logger.error("[Exception] " + str(e))
        processing_article["valid"] = False
        
    return processing_article

# process_news_piece(arquivo_response[223])

In [224]:

arquivo_response = []

# In total 588 Publico news articles.
arquivo_response += search_arquivo_news('', datetime(2018,10,1), date.today(), ['publico.pt noticia -js ', 'acervo.publico.pt noticia -js ']) 

# In total 553 Expresso news articles.
arquivo_response += search_arquivo_news('', datetime(2018,10,1), date.today(), ['expresso.pt 2018', 'expresso.pt 2019', 'expresso.pt 2020', 'expresso.sapo.pt 2018', 'expresso.sapo.pt 2019', 'expresso.sapo.pt 2020'])

# In total 37 DN news articles.
arquivo_response += search_arquivo_news('', datetime(2018,10,1), date.today(), ['www.dn.pt politica', 'www.dn.pt sociedade', 'dn.sapo.pt politica', 'dn.sapo.pt sociedade'])

# In total 101 DN news articles.
arquivo_response += search_arquivo_news('', datetime(2018,10,1), date.today(), ['visao.sapo.pt 2018', 'visao.sapo.pt 2019', 'visao.sapo.pt 2020'])

print("In total", len(arquivo_response), "arquivo items. The first 5 items are:\n")
print(arquivo_response[:5])

2021-04-20 01:25:31.430 | INFO     | __main__:query_arquivo:37 - [FETCHING] https://arquivo.pt/textsearch?q=%22%22&from=20181001000000&to=20210420000000&siteSearch=publico.pt+noticia+-js+%2Cacervo.publico.pt+noticia+-js+&fields=title%2Ctstamp%2CoriginalURL%2ClinkToNoFrame&type=html&maxItems=2000&itemsPerSite=1000
2021-04-20 01:25:33.552 | INFO     | __main__:query_arquivo:37 - [FETCHING] https://arquivo.pt/textsearch?q=%22%22&from=20181001000000&to=20210420000000&siteSearch=expresso.pt+2018%2Cexpresso.pt+2019%2Cexpresso.pt+2020%2Cexpresso.sapo.pt+2018%2Cexpresso.sapo.pt+2019%2Cexpresso.sapo.pt+2020&fields=title%2Ctstamp%2CoriginalURL%2ClinkToNoFrame&type=html&maxItems=2000&itemsPerSite=333
2021-04-20 01:26:03.646 | ERROR    | utils:try_request:34 - [HTTPSConnectionPool(host='arquivo.pt', port=443): Read timed out. (read timeout=30)] for [{'q': '""', 'from': '20181001000000', 'to': '20210420000000', 'siteSearch': 'expresso.pt 2018,expresso.pt 2019,expresso.pt 2020,expresso.sapo.pt 2018,

In total 1279 arquivo items. The first 5 items are:

[{'title': 'Praga de baratas “grandes” aflige Parque das Nações | Lisboa | PÚBLICO', 'originalURL': 'http://publico.pt/2019/08/29/local/noticia/baratas-parque-nacoes-1884816', 'tstamp': '20190831150457', 'linkToNoFrame': 'https://arquivo.pt/noFrame/replay/20190831150457/http://publico.pt/2019/08/29/local/noticia/baratas-parque-nacoes-1884816'}, {'title': 'Mais de 50 crianças adoptadas foram devolvidas nos últimos três anos | Adopção | PÚBLICO', 'originalURL': 'http://publico.pt/2019/07/30/sociedade/noticia/ultimos-tres-anos-devolvidas-53-criancas-adopcao-1881728', 'tstamp': '20190831150900', 'linkToNoFrame': 'https://arquivo.pt/noFrame/replay/20190831150900/http://publico.pt/2019/07/30/sociedade/noticia/ultimos-tres-anos-devolvidas-53-criancas-adopcao-1881728'}, {'title': 'Empresas de Portugal e Galiza suspeitas de se apropriarem de 20 milhões de fundos europeus | Justiça', 'originalURL': 'http://publico.pt/2019/08/03/sociedade/notic

In [225]:
# This script takes about 7 mins to run 
processed = []

logger.info("[START PROCESSING]")
for arquivo_item in tqdm(arquivo_response):
    logger.info("[PROCESS] Article %s" % arquivo_item['linkToNoFrame'])
    processed.append(process_news_piece(arquivo_item))
    
# Pickle processed news
with open('processed.pkl', 'wb') as f:
    pickle.dump(processed, f)
    
def default(o):
    if isinstance(o, (datetime)):
        return o.isoformat()
    
# Also output it to a JSON file
with open('processed.json', 'w') as outfile:
    json.dump(processed, outfile, default=default)

2021-04-20 01:27:27.331 | INFO     | __main__:<module>:3 - [START PROCESSING]


2021-04-20 01:27:27.411 | INFO     | __main__:<module>:5 - [PROCESS] Article https://arquivo.pt/noFrame/replay/20190831150457/http://publico.pt/2019/08/29/local/noticia/baratas-parque-nacoes-1884816
2021-04-20 01:27:29.265 | INFO     | __main__:<module>:5 - [PROCESS] Article https://arquivo.pt/noFrame/replay/20190831150900/http://publico.pt/2019/07/30/sociedade/noticia/ultimos-tres-anos-devolvidas-53-criancas-adopcao-1881728
2021-04-20 01:27:29.848 | INFO     | __main__:<module>:5 - [PROCESS] Article https://arquivo.pt/noFrame/replay/20190831150704/http://publico.pt/2019/08/03/sociedade/noticia/espanha-descobre-fraude-fundos-europeus-galiza-portugal-1882260
2021-04-20 01:27:30.425 | INFO     | utils:is_duplicate:198 - DUP [Unidade de Polícia Judiciária Orgânica da Guarda Civil de Pontevedra, Polícia Judiciária, s1=0.20, s2=1.00, ji=0.20] : Polícia Judiciária
2021-04-20 01:27:30.456 | INFO     | __main__:<module>:5 - [PROCESS] Article https://arquivo.pt/noFrame/replay/20190905204112/htt

2021-04-20 01:27:43.768 | INFO     | utils:is_duplicate:198 - DUP [Lisboa Stargate, Lisboa, s1=0.50, s2=1.00, ji=0.50] : Lisboa
2021-04-20 01:27:43.774 | INFO     | __main__:<module>:5 - [PROCESS] Article https://arquivo.pt/noFrame/replay/20190325230526/https://acervo.publico.pt/fugas/noticia/moro-numa-ilha-subtropical-1733129
2021-04-20 01:27:44.066 | ERROR    | __main__:process_news_piece:76 - [Exception] News piece is too long 19977
2021-04-20 01:27:44.068 | INFO     | __main__:<module>:5 - [PROCESS] Article https://arquivo.pt/noFrame/replay/20190314164023/https://acervo.publico.pt/culturaipsilon/noticia/comissao-portuguesa-das-sepulturas-de-guerra-1668608
2021-04-20 01:27:44.376 | INFO     | __main__:<module>:5 - [PROCESS] Article https://arquivo.pt/noFrame/replay/20190314164036/https://acervo.publico.pt/culturaipsilon/noticia/estatua-do-soldado-desconhecido-de-lamego-1667769
2021-04-20 01:27:44.627 | INFO     | __main__:<module>:5 - [PROCESS] Article https://arquivo.pt/noFrame/rep

2021-04-20 01:27:56.392 | ERROR    | __main__:process_news_piece:76 - [Exception] News piece is too long 33481
2021-04-20 01:27:56.395 | INFO     | __main__:<module>:5 - [PROCESS] Article https://arquivo.pt/noFrame/replay/20190925120908/https://acervo.publico.pt/politica/noticia/um-ano-visto-por-antonio-barreto-1680515
2021-04-20 01:27:56.655 | INFO     | utils:is_duplicate:198 - DUP [António Barreto, Barreto, s1=0.50, s2=1.00, ji=0.50] : Barreto
2021-04-20 01:27:56.660 | INFO     | utils:organize_entities:259 - FDUP [Barreto, António Barreto] : Barreto
2021-04-20 01:27:56.662 | INFO     | __main__:<module>:5 - [PROCESS] Article https://arquivo.pt/noFrame/replay/20190831142601/http://publico.pt/2019/06/01/fugas/noticia/resende-filao-cerejas-1874564
2021-04-20 01:27:57.196 | INFO     | __main__:<module>:5 - [PROCESS] Article https://arquivo.pt/noFrame/replay/20190829090949/https://acervo.publico.pt/mundo/noticia/--e-o-ritmo-e-de-boi---1687742
2021-04-20 01:27:57.633 | INFO     | __main_

2021-04-20 01:28:09.507 | INFO     | __main__:<module>:5 - [PROCESS] Article https://arquivo.pt/noFrame/replay/20190831120017/http://publico.pt/2019/08/30/culturaipsilon/noticia/casorio-legendary-tigerman-calcuta-1884843
2021-04-20 01:28:09.950 | INFO     | __main__:<module>:5 - [PROCESS] Article https://arquivo.pt/noFrame/replay/20190905204955/http://publico.pt/2019/07/10/ciencia/noticia/sairam-africa-cedo-pensava-1879323
2021-04-20 01:28:10.371 | INFO     | __main__:<module>:5 - [PROCESS] Article https://arquivo.pt/noFrame/replay/20190831135056/http://publico.pt/2019/08/23/culto/noticia/cao-companhia-saude-estudo-1884263
2021-04-20 01:28:10.798 | INFO     | utils:is_duplicate:198 - DUP [Mayo Clinic Proceedings, Mayo Clinic, s1=0.67, s2=1.00, ji=0.67] : Mayo Clinic
2021-04-20 01:28:10.803 | INFO     | __main__:<module>:5 - [PROCESS] Article https://arquivo.pt/noFrame/replay/20190902112541/https://acervo.publico.pt/fugas/noticia/ilhas-o-cafe-da-faja-dos-vimes-1733057
2021-04-20 01:28:1

...

2021-04-20 01:33:50.236 | INFO     | __main__:<module>:5 - [PROCESS] Article https://arquivo.pt/noFrame/replay/20190824192550/https://www.dn.pt/portugal/interior/amp/almaraz-europa-precisa-de-350-mil-milhoes-de-euros-para-desmantelar-91-centrais---carlos-zorrinho-8587912.html
2021-04-20 01:33:50.455 | INFO     | utils:is_duplicate:198 - DUP [Idanha-a-Nova, Câmara de Idanha-a-Nova, s1=1.00, s2=0.33, ji=0.33] : Câmara de Idanha-a-Nova
2021-04-20 01:33:50.461 | INFO     | __main__:<module>:5 - [PROCESS] Article https://arquivo.pt/noFrame/replay/20181208191145/https://www.dn.pt/mundo/interior/xi-jinping-a-china-vai-aderir-sempre-ao-principio-do-respeito-mutuo-10279949.html
2021-04-20 01:33:50.839 | INFO     | __main__:<module>:5 - [PROCESS] Article https://arquivo.pt/noFrame/replay/20181208191831/https://www.dn.pt/cultura/interior/thom-yorke-atua-a-solo-no-nos-alive-10290218.html
2021-04-20 01:33:51.227 | INFO     | utils:is_duplicate:198 - DUP [Thom Yorke, Yorke, s1=0.50, s2=1.00, ji=0.50

2021-04-20 01:33:58.731 | INFO     | utils:is_duplicate:198 - DUP [Assembleia Geral das Nações Unidas, Assembleia Geral, s1=0.40, s2=1.00, ji=0.40] : Assembleia Geral
2021-04-20 01:33:58.736 | INFO     | __main__:<module>:5 - [PROCESS] Article https://arquivo.pt/noFrame/replay/20181209041953/https://www.dn.pt/default.aspx?referrer=www.globalimagens.pt
2021-04-20 01:33:59.296 | INFO     | utils:is_duplicate:198 - DUP [Assembleia Geral das Nações Unidas, Assembleia Geral, s1=0.40, s2=1.00, ji=0.40] : Assembleia Geral
2021-04-20 01:33:59.302 | INFO     | __main__:<module>:5 - [PROCESS] Article https://arquivo.pt/noFrame/replay/20191231204142/https://visao.sapo.pt/fotografia/2018-04-24-world-press-photo-2018/
2021-04-20 01:33:59.494 | INFO     | __main__:<module>:5 - [PROCESS] Article https://arquivo.pt/noFrame/replay/20181025224437/https://visao.sapo.pt/exame/2018-03-28-Banco-de-Portugal-ve-desemprego-abaixo-dos-6-em-2020
2021-04-20 01:33:59.787 | INFO     | __main__:<module>:5 - [PROCESS

2021-04-20 01:34:06.895 | INFO     | __main__:<module>:5 - [PROCESS] Article https://arquivo.pt/noFrame/replay/20191218020320/https://visao.sapo.pt/opiniao/2019-04-23-o-circunstancial-vs-o-essencial/
2021-04-20 01:34:07.149 | INFO     | __main__:<module>:5 - [PROCESS] Article https://arquivo.pt/noFrame/replay/20190329051651/https://visao.sapo.pt/exame/2019-03-28-Portugal-volta-a-comprar-mais-do-que-vende-ao-exterior
2021-04-20 01:34:07.413 | INFO     | __main__:<module>:5 - [PROCESS] Article https://arquivo.pt/noFrame/replay/20191204191220/https://visao.sapo.pt/visaojunior/2019-11-08-o-que-vamos-comer-no-futuro/
2021-04-20 01:34:07.632 | INFO     | __main__:<module>:5 - [PROCESS] Article https://arquivo.pt/noFrame/replay/20191205082247/https://visao.sapo.pt/cultura/2019-03-04-morreu-o-vocalista-dos-the-prodigy/
2021-04-20 01:34:07.841 | INFO     | utils:is_duplicate:198 - DUP [Liam Howlett, Howlett, s1=0.50, s2=1.00, ji=0.50] : Howlett
2021-04-20 01:34:07.846 | INFO     | __main__:<mod

2021-04-20 01:34:13.312 | INFO     | __main__:<module>:5 - [PROCESS] Article https://arquivo.pt/noFrame/replay/20191106231228/http://visao.sapo.pt/exame/2019-10-24-Belga-Xior-compra-residencias-de-estudantes-a-U.hub-por-130-milhoes-de-euros
2021-04-20 01:34:13.535 | INFO     | __main__:<module>:5 - [PROCESS] Article https://arquivo.pt/noFrame/replay/20191205073308/https://visao.sapo.pt/sociedade/2019-06-09-pusemos-um-walkman-nas-maos-de-miudos-e-aconteceu-isto/
2021-04-20 01:34:13.742 | INFO     | __main__:<module>:5 - [PROCESS] Article https://arquivo.pt/noFrame/replay/20191129202651/https://visao.sapo.pt/visaobs/2019-06-05-poupar-na-conta-da-luz-com-estilo-1/
2021-04-20 01:34:13.950 | INFO     | utils:is_duplicate:198 - DUP [LG Electronics, LG, s1=0.50, s2=1.00, ji=0.50] : LG
2021-04-20 01:34:13.955 | INFO     | __main__:<module>:5 - [PROCESS] Article https://arquivo.pt/noFrame/replay/20191204194427/https://visao.sapo.pt/mundo/2019-03-27-Adiamento-do-Brexit-aprovado-mas-impasse-conti

2021-04-20 01:34:19.378 | INFO     | __main__:<module>:5 - [PROCESS] Article https://arquivo.pt/noFrame/replay/20191228071800/https://visao.sapo.pt/visaose7e/ver/2019-10-27-woody-allen-fora-de-horas-no-filme-um-dia-de-chuva-em-nova-iorque/
2021-04-20 01:34:19.623 | INFO     | __main__:<module>:5 - [PROCESS] Article https://arquivo.pt/noFrame/replay/20191205045904/https://visao.sapo.pt/visaose7e/sair/2019-10-25-este-fim-de-semana-ha-jardins-abertos-em-lisboa-e-as-visitas-sao-gratis/
2021-04-20 01:34:19.839 | INFO     | utils:is_duplicate:198 - DUP [Lisboa, Câmara Municipal de Lisboa, s1=1.00, s2=0.25, ji=0.25] : Lisboa
2021-04-20 01:34:19.844 | INFO     | __main__:<module>:5 - [PROCESS] Article https://arquivo.pt/noFrame/replay/20191205051750/https://visao.sapo.pt/visaose7e/livros-e-discos/2019-09-23-pixies-estao-finalmente-de-volta-com-o-album-beneath-the-eyrie/
2021-04-20 01:34:20.073 | INFO     | __main__:<module>:5 - [PROCESS] Article https://arquivo.pt/noFrame/replay/20191218045348

In [211]:
change_entities = {
    'PER': ('Carlos Carvalhal', 'Fábio Oliveira'),
    'LOC': ('Famalicão', 'FC Mourinha') # ner ... pls 
}


with open('processed.json') as json_file:
    processed = json.load(json_file)
    
def replace_text(text, new_text, initial_char, final_char):
    return text[:initial_char] + new_text + text[final_char:]

def replace_appearances(text, entities, label, entity_before, entity_after):
    if label in entities.keys() and entity_before in entities[label].keys():
        appearances = entities[label][entity_before]
        for initial_char, final_char in appearances:
            text = replace_text(text, entity_after, initial_char, final_char)
    return text
    
def mix_entities(text, entities, change_entities):
    for label, (entity_before, entity_after) in change_entities.items():
        replace_appearances(text, entities, label, entity_before, entity_after)
    return text

article = processed[223]
article['summary'] = mix_entities(article['summary'], article['summary_entities'], change_entities)
article['title'] = mix_entities(article['title'], article['title_entities'], change_entities)
print(article)

def default(o):
    if isinstance(o, (datetime)):
        return o.isoformat()
with open('article.json', 'w') as outfile:
    json.dump(article, outfile, default=default)


{'title': 'Famalicão soma nova vitória e lidera à condição', 'original_title': 'Famalicão soma nova vitória e lidera à condição | Futebol | PÚBLICO', 'original_url': 'http://publico.pt/2019/08/16/desporto/noticia/famalicao-soma-nova-vitoria-lidera-condicao-1883636', 'url': 'https://arquivo.pt/noFrame/replay/20190831132137/http://publico.pt/2019/08/16/desporto/noticia/famalicao-soma-nova-vitoria-lidera-condicao-1883636', 'timestamp': '2019-08-31T13:21:37', 'website': 'publico.pt', 'text': 'Uma bomba do defesa Patrick William, na cobrança de um livre directo (66’), garantiu esta sexta-feira, ante o Rio Ave, a segunda vitória (1-0) do Famalicão e a liderança provisória do campeonato, com seis pontos em dois jogos e sem qualquer golo sofrido. No regresso a casa, na condição de primodivisionário, 25 anos depois do último encontro disputado entre a elite, os famalicenses superaram o Rio Ave de Carlos Carvalhal, que fez a estreia na prova.\n\nO equilíbrio imperou numa noite marcada por alguma

In [115]:
def get_most_frequent_entity_label(processed_list, label, topn=None):
    all_ents = itertools.chain(*[p['summary_entities'][label].keys() for p in processed_list if p['valid'] and label in p['summary_entities'].keys()])
    return Counter(all_ents).most_common(topn)

def get_most_frequent_enteties(processed_list, topn=None):
    most_frequent_entities = dict()
    for label in ['PER', 'LOC', 'ORG', 'MISC']:
        most_frequent_entities[label] = get_most_frequent_entity_label(processed_list, label, topn=10)
    return most_frequent_entities

def get_weighted_random_entity(label, most_frequent_entities):
    return random.choices(*zip(*most_frequent_entities[label]))[0]

most_frequent_enteties = get_most_frequent_enteties(processed, 10)

for label in ['PER', 'LOC', 'ORG', 'MISC']:
    print(label, ': ', get_weighted_random_entity(label, most_frequent_enteties), '\n')

PER :  António Costa 

LOC :  Europa 

ORG :  Lusa 

MISC :  Subscreva 



In [230]:
keys = ["title", "original_title", "url", "original_url", "image", "timestamp", "summary", "fake_details"]
fake = []

for p in processed[:5]:
    if not p['valid']: continue
    
    try:
        # Order by number of occurrences  
        entity_count_per_label = { key: Counter(value.keys()) for key, value in p['summary_entities'].items() }
        
        entities_replaced = []
        for label in ['PER']: #, 'LOC', 'ORG', 'MISC']:
            if label not in entity_count_per_label.keys():
                raise Exception('Zero entities with label ' + label)
            chosen_entity = list(entity_count_per_label[label].keys())[0]
            new_entity = get_weighted_random_entity(label, most_frequent_enteties)
            entities_replaced.append({
                "original_entity_name": chosen_entity,
                "replaced_entity_name": new_entity
            })
            img_url = get_random_arquivo_image(new_entity, show=True)
            print(label, '::', chosen_entity, '=>', new_entity)
    #         entity_count = list(dict(entitie_count[label].most_common()).keys())
    #         print(entity_count)


        p["fake_details"] = {
            "fake_title":  replace_appearances(p['title'], p['title_entities'], 'PER', chosen_entity, new_entity),
            "fake_summary":  replace_appearances(p['summary'], p['summary_entities'], 'PER', chosen_entity, new_entity), 
            "fake_image_url": img_url,
            "entities_replaced": entities_replaced
        }
        p = dict((k, p[k]) for k in keys if k in p.keys())
        
    except Exception as e:
        print(e)
        p['fake_details'] = None
    
    fake.append(p)

2021-04-20 01:40:22.784 | INFO     | __main__:query_arquivo:37 - [FETCHING] https://arquivo.pt/imagesearch?q=%22%E2%80%98Brexit%E2%80%99%22&from=20181001000000&to=20210420000000


2021-04-20 01:40:22.920 | INFO     | __main__:query_arquivo:37 - [FETCHING] https://arquivo.pt/imagesearch?q=%22May%22&from=20181001000000&to=20210420000000


PER :: Elisa Soares => ‘Brexit’


2021-04-20 01:40:23.078 | INFO     | __main__:query_arquivo:37 - [FETCHING] https://arquivo.pt/imagesearch?q=%22Brexit%22&from=20181001000000&to=20210420000000


PER :: Rute Agulhas => May


2021-04-20 01:40:23.188 | INFO     | __main__:query_arquivo:37 - [FETCHING] https://arquivo.pt/imagesearch?q=%22M%C3%A1rio+Centeno%22&from=20181001000000&to=20210420000000


PER :: Frangus => Brexit


PER :: João Gomes Cravinho => Mário Centeno
Zero entities with label PER
